In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline
mnist = input_data.read_data_sets('data/mnist/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


In [ ]:
trainimg = mnist.train.images
trainlabel = mnist.train.labels
randidx = np.random.randint(trainimg.shape[0], size=2)

def get_batch(i, batch_size, input_var):
    if batch_size > input_var.shape[0]:
        return input_var
    start = (i*batch_size)%input_var.shape[0]
    overflow = start + batch_size - input_var.shape[0]
    if overflow <= 0:
        return input_var[start:start+batch_size]
    else:
        return np.r_[input_var[start:], input_var[:overflow]]
    
def build_cnn(cnn_shape, patch_shape, X):
    n_before = int(X.get_shape()[3])
    layer = X
    for idx, val in enumerate(cnn_shape):
        W = tf.Variable(tf.truncated_normal([patch_shape[0], patch_shape[1], n_before, val], stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[val]))
        layer = tf.nn.relu(tf.nn.conv2d(layer, W, strides=[1, 1, 1, 1], padding='SAME') + b)
        layer = tf.nn.max_pool(layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        n_before = val
    layer_shape = layer.get_shape().as_list()
    n_out = layer_shape[1] * layer_shape[2] * layer_shape[3]
    return tf.reshape(layer, [-1, n_out])
    
def build_nn(shape, X, Y):
    n_before = int(X.get_shape()[1])
    n_out = int(Y.get_shape()[1])
    layer = X
    for idx, val in enumerate(shape):
        W = tf.Variable(tf.truncated_normal([n_before, val], stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[val]))
        layer = tf.nn.relu(tf.matmul(layer, W)+b)
        n_before = val
    W = tf.Variable(tf.truncated_normal([n_before, n_out], stddev=0.1))
    b = tf.Variable(tf.constant(0.1, shape=[n_out]))
    return tf.matmul(layer, W)+b

print("function loaded")

function loaded


In [ ]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

mask_weight = tf.Variable(tf.ones(X.get_shape()[1]))
mask_layer = tf.nn.relu(tf.mul(X, mask_weight))
cnn_layer = build_cnn([16, 32], [5,5], tf.reshape(mask_layer, [-1, 28, 28, 1]))
dense_layer = build_nn([512], cnn_layer, Y)
keep_prob = tf.placeholder(tf.float32)
dropout = tf.nn.dropout(dense_layer, keep_prob)
hypothesis = tf.nn.softmax(dropout)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=[1]))

learning_rate = 0.001
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

trainsize = mnist.train.num_examples
batchsize = 30
batch_per_epoch = int(trainsize/batchsize)
print ("Training %d, mini-batch %d * %d" % (trainsize, batchsize, batch_per_epoch))

correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(hypothesis,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for i in range(batch_per_epoch*5):
    if i % batch_per_epoch == 0 :
        print ("%6dth epoch : test accuracy = %.3f" % \
               (i / batch_per_epoch, sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1})))
        
    batch_x = get_batch(i, batchsize, trainimg)
    batch_y = get_batch(i, batchsize, trainlabel)
    sess.run(train, feed_dict={X:batch_x, Y:batch_y, keep_prob:0.5})
    
train_accuracy = 0
for i in range(batch_per_epoch):
    batch_x = get_batch(i, batchsize, trainimg)
    batch_y = get_batch(i, batchsize, trainlabel)
    train_accuracy += sess.run(accuracy, feed_dict={X:batch_x, Y:batch_y, keep_prob:1})
        
print ("train complete : test accuracy = %.3f" % (sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1})))
print ("                 train accuracy = %.3f" % (train_accuracy / batch_per_epoch))

Training 55000, mini-batch 30 * 1833
     0th epoch : test accuracy = 0.071


In [ ]:
mask = sess.run(mask_weight).reshape(28,28)
plt.matshow(mask, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)